**Rich-Get-Richer from an economic perspective**

Imagine that a large group of people are given an amount of 100 euros each, under the condition that they must keep it invested in a rather volatile and unproductive stockmarket. Per time period, each person independently makes their own investments (they could invest a certain percentage of their money, ranging from 0 to 100%), with an average return of 0%. This means the following: each time period half of the population (randomly chosen) has a gain of p% (capital multiplied by 1 + p) and the other half of the people experiences a reduction of capital with factor 1/(1 + p), so the total amount of capital stays the same.

Authors: 
Vladimir Halchenko (s2426447) and Ilya Sinyugin (s2586967) 

In [3846]:
from matplotlib import pyplot
import numpy
import pandas
import random
import math
from typing import Callable, Any, List, Final
from decimal import Decimal

In [3847]:
def gain(a: float, p: float) -> float:
    '''
    Model (in the general form):
    a is an initial capital.
    p is a %. p є [0; 1]
    '''
    if not(p >= 0 and p <= 1):
        raise Exception("Wrong arguments provided!")
    else:
        a = a * (1 + p)
        return a



def loss(a: float, p: float) -> float:
    '''
    Model (in the general form):
    a is an initial capital.
    p is a %. p є [0; 1]
    '''
    if not(p >= 0 and p <= 1):
        raise Exception("Wrong arguments provided!")
    else:
        a = a * 1/(1+p)
        return a


capital = [100]*500
def run_model(years: int) -> List[float]:
    '''
    Нears is a number of years to run the model for. 1 year = 1 time period
    '''
    for i in range(years):
        # Shuffle the list of population's capital 
        random.shuffle(capital)
        # pick out the first 50 as winners and the last 50 as losers
        winners = capital[:int(len(capital)/2)]
        losers = capital[int(len(capital)/2):]
        for index in range(int(len(capital)/2)):
            # interest rate of 12%
            capital[index] = gain(winners[index], 0.12)
            capital[index + int(len(capital)/2)] = loss(losers[index], 0.12)
    return capital

In [3848]:
YEARS = 65

In [3849]:
# Code taken from Weeks 3-4 project (question 5) viewcount_distribution.py and adapted 
# to the current model. 
def plot_powerlaw_linear(capital: List[Decimal], filename: str) -> None:
    '''
    Plot the distribution of the capital against the number of people
    '''
    histogram, edges = numpy.histogram(capital, bins=18, density=False)
    pyplot.xlabel('Number of people')
    pyplot.ylabel('Capital')
    pyplot.plot(histogram, edges[:-1])
    pyplot.savefig(filename)
    pyplot.clf()

def plot_log(capital: List[Decimal], filename: str) -> None:
    '''
    Plot the distribution of the log of capital against the log of the number of people
    '''
    histogram, edges = numpy.histogram(capital, density=False)
    pyplot.xlabel('Log of number of people')
    pyplot.ylabel('Log of capital')
    # the for loop is used to avoid any -inf in the model, which occur by taking a log 
    # of a very small number, close to 0    
    for i in range(len(histogram)):
        histogram[i] += 1
    #remove the only extreme outlier, that shall be ignored
    histogram = histogram[1:]
    edges = edges[1:]
    log_histogram = numpy.log10(histogram)
    # remove the last bin edge to match the histogram size
    log_bin_edges = numpy.log10(edges[:-1]) 
    pyplot.scatter(log_histogram, log_bin_edges)
    # plot a line of best fit 
    gradient, intercept = numpy.polyfit(log_histogram, log_bin_edges, 1)
    pyplot.plot(log_histogram, gradient*log_histogram + intercept, linestyle="dashed", linewidth=2)
    pyplot.savefig(filename)
    pyplot.clf()

In [3850]:
# Code taken from Weeks 3-4 project (question 5) viewcount_distribution.py and adapted
# to the current model.
def plot_powerlaw_linear_percentage(capital: List[Decimal], filename: str) -> None:
    '''
    Plot the distribution of the percentage of capital against the percentage of people
    '''
    total = math.fsum(capital)
    percentages = [x/total*100 for x in capital]
    histogram, edges = numpy.histogram(percentages, bins=13, density=True)
    histogram = [x/len(capital)*100 for x in histogram]
    pyplot.xlabel('Percentage of people')
    pyplot.ylabel('Percentage of capital')
    pyplot.plot(histogram, edges[:-1])
    pyplot.savefig(filename)
    pyplot.clf()

In [3851]:
plot_powerlaw_linear(run_model(YEARS), "powerlaw_linear.jpg")
plot_log(capital, "log.jpg")
plot_powerlaw_linear_percentage(capital, "powerlaw_linear_percentage.jpg")

<Figure size 432x288 with 0 Axes>

In [3852]:
# Progressive Income Tax Model 
def gain_with_tax(a: float, p: float, capital: List[float]) -> float: 
    '''
    Model (in the general form):
    a is an initial capital.
    p is a %. p є [0; 1]
    '''
    if not(p >= 0 and p <= 1):
        raise Exception("Wrong arguments provided!")
    else:
        # make a list of capitals in the descending order 
        capital_sorted = sorted(capital, reverse=True)
        index = capital_sorted.index(a)
        # not taxing those, who lost significant amount of their investments
        if a < 100: 
            tax = 0
        else: 
            # check the index of the capital in the sorted list
            # from richest to poorest
            tax = min((1-(index/len(capital)))*0.05, 0.05)
        a = a * (1+p) * (1-tax)
        return max(a, 0)
        
# New Model with income tax 
def run_model_with_tax(years: int) -> List[float]:
    '''
    years is a number of years to run the model for. 1 year = 1 time period
    '''
    for i in range(years):
        # Shuffle the list of population's capital 
        random.shuffle(capital)
        # pick out the first 50 as winners and the last 50 as losers
        winners = capital[:int(len(capital)/2)]
        losers = capital[int(len(capital)/2):]
        for index in range(int(len(capital)/2)):
            # interest rate of 12%
            capital[index] = gain_with_tax(winners[index], 0.12, capital)
            capital[index + int(len(capital)/2)] = loss(losers[index], 0.12)
    return capital

def plot_log_tax(capital: List[Decimal], filename: str) -> None:
    '''
    Plot the distribution of the log of capital against the log of the number of people
    '''
    histogram, edges = numpy.histogram(capital, density=False)
    pyplot.xlabel('Log of number of people')
    pyplot.ylabel('Log of capital')
    for i in range(len(histogram)):
        histogram[i] += 1
    #remove an extreme outlier 
    histogram = histogram[1:]
    edges = edges[1:]
    log_histogram = numpy.log10(histogram)
    log_bin_edges = numpy.log10(edges[:-1]) # since it returns an extra bin edge
    pyplot.scatter(log_histogram, log_bin_edges)
    # Here a line of best fit is not plotted, since it is observable by the histogram 
    # that the distribution is not strictly linear
    pyplot.savefig(filename)
    pyplot.clf()


capital = [100]*500
plot_powerlaw_linear(run_model_with_tax(YEARS), "powerlaw_linear_with_tax.jpg")
plot_powerlaw_linear_percentage(capital, "powerlaw_linear_percentage_with_tax.jpg")
plot_log_tax(capital, "log_with_tax.jpg")

<Figure size 432x288 with 0 Axes>